# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [59]:
import requests
import pandas as pd
import json
import datetime
from IPython.display import JSON

API_key ='fsq3bdYof248uF9howPsH7I8nebVs+VO6OzmKCU+/6uUoxw='

dfs = pd.read_json('citybike_station.json')

#Set API endpoint and parameters
url = 'https://api.foursquare.com/v3/places/nearby'

list_df = []

for index, row in dfs.iterrows():
    name = row['station_name']
    latitude = row['latitude']
    longitude = row['longitude']
    #print(f'{latitude},{longitude}')
    
    # Set location parameters for each station
    params = {
        'll': f'{latitude},{longitude}',
        'radius': 1000,
        'query': 'Italian restaurant',
        'rating':'5',
    }

    headers = {"Authorization": API_key
    }

    # Send API request and parse JSON response
    
    response_fsq = requests.get(url, params=params, headers=headers)
    response_fsq_json = response_fsq.json()
    #print("staion:", response_fsq_json)
 
# # Create an empty list, where we'll have one item per row
# list_df = []

# Loop through whatever you're trying to make a rows out of
    for i in range(len(response_fsq_json['results'])):

        # For each row, grab the values you want to make columns out of
        name_col = response_fsq_json['results'][i]['name']
        link_col = response_fsq_json['results'][i]['link']
        

        # Append a dictionary to our list, where keys represent column names and values represent the values for those columns
        list_df.append({'name': name_col, 'link': link_col, 'station_name' : name, 'latitude' : latitude, 'longitude' : longitude})

   
    
# Turn the list of dictionaries into a dataframe
df = pd.DataFrame(list_df)
df
    
JSON(response_fsq_json)    
    
# #response_fsq_json['results'][0]['name']   
# response_fsq_json['results'][0]['geocodes']['link'] 


<IPython.core.display.JSON object>

In [57]:
df 

,name,link,station_name,latitude,longitude
0,Verace Italian Restaurant,/v3/places/61b4e83e88049e706c18cd7c,Bayfront Park,43.269288,-79.871327
1,Hamilton Italian Recreation Club,/v3/places/4d0d35b195c1a1cd8ef000ff,Bayfront Park,43.269288,-79.871327
2,The Express Italian Restaurant,/v3/places/5b394376c66666002c5e54ef,Bayfront Park,43.269288,-79.871327
3,Ora Italian Kitchen + Bar,/v3/places/51ba738f7dd2b8c55fbf7611,Bayfront Park,43.269288,-79.871327
4,Nino's Authentic Italian Restaurant,/v3/places/55bcde11498e244a3a859792,Bayfront Park,43.269288,-79.871327
...,...,...,...,...,...
1130,Italian Vice Consulate,/v3/places/4e4fdc2f18383eb2af270fa6,Ferguson at King William,43.255179,-79.861443
1131,The Express Italian Restaurant,/v3/places/5b394376c66666002c5e54ef,Ferguson at King William,43.255179,-79.861443
1132,Tatemono Sushi Bar & Restaurant,/v3/places/55c6384b498e1717d76b4cb4,Ferguson at King William,43.255179,-79.861443
1133,One Duke Restaurant & Lounge,/v3/places/4b6a4bc5f964a52024d12be3,Ferguson at King William,43.255179,-79.861443


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [7]:
import json

data = response_fsq_json
poi_list = data['results']

for poi in poi_list:
    name = poi['name']
    rating = poi.get('rating', 'N/A')
    location = poi['location']
    link = poi['link']

    # Process the extracted details as needed
    print(f"Name: {name}")
    print(f"Location: {location}")
    print(f"Link: {link}")
    print(f"Rating: {rating}")
    

Name: Verace Italian Restaurant
Location: {'address': '312 Lakeshore Rd E', 'country': 'CA', 'cross_street': 'Reynolds Street', 'formatted_address': '312 Lakeshore Rd E (Reynolds Street), Oakville ON L6J 1J2', 'locality': 'Oakville', 'postcode': 'L6J 1J2', 'region': 'ON'}
Link: /v3/places/61b4e83e88049e706c18cd7c
Rating: N/A
Name: Italian Vice Consulate
Location: {'address': '105 Main St E', 'country': 'CA', 'cross_street': '', 'formatted_address': '105 Main St E, Hamilton ON L8N 1G6', 'locality': 'Hamilton', 'postcode': 'L8N 1G6', 'region': 'ON'}
Link: /v3/places/4e4fdc2f18383eb2af270fa6
Rating: N/A
Name: The Express Italian Restaurant
Location: {'address': '1477 Lakeshore Rd', 'country': 'CA', 'cross_street': '', 'formatted_address': '1477 Lakeshore Rd, Burlington ON L7S 1B5', 'locality': 'Burlington', 'postcode': 'L7S 1B5', 'region': 'ON'}
Link: /v3/places/5b394376c66666002c5e54ef
Rating: N/A
Name: One Duke Restaurant & Lounge
Location: {'address': '1 Duke St', 'country': 'CA', 'cro

Put your parsed results into a DataFrame

In [34]:
businesses = []
for business in poi_list:
     name = business['name']
     distance = business['distance']
     #review_count = business['review_count']

     businesses.append({
             'Name' : name,
             'Distance' : distance,
             'Rating' : rating
             
     })
df_foursquare = pd.DataFrame(businesses) 
df_foursquare
# missing the columns lat/long from station

,Name,Distance,Rating
0,Verace Italian Restaurant,26638,N/A
1,Italian Vice Consulate,736,N/A
2,The Express Italian Restaurant,9304,N/A
3,One Duke Restaurant & Lounge,921,N/A
4,Tatemono Sushi Bar & Restaurant,301,N/A
5,Oriental Restaurant & Tavern,385,N/A


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [69]:
import requests
import pandas as pd
from IPython.display import JSON


yelp_df = pd.DataFrame()
yelp_api_key = "Bearer EGDOETRpkOPCnv9PVI0sg9WC9MVbib8fGzoWpHcQL0JnwicA7GGUjUoDjk4NjR3_Ev8Fnr-5zchkoThzsI7U1ncpiKXEZnzGNQSby-dah6xaGxfltSAPlkw9L3B6ZHYx"

# Set API headers

yelp_headers = {"accept": "application/json"}
yelp_headers['Authorization'] = yelp_api_key

businesses = []

for index, row in dfs.iterrows(): #city_bikes
    
    stname = row['station_name']
    latitude = row['latitude']
    longitude = row['longitude']
    
    #url
    
    yelp_url = "https://api.yelp.com/v3/businesses/search?latitude=" + str(latitude) + "&longitude=" + str(longitude) +"&term=italian%20restaurant&radius=1000&sort_by=best_match&limit=20"


    # Send API request and parse JSON response
    
    response_yelp = requests.get(yelp_url, headers=yelp_headers)
    
    # print("Response Status Code:", response_yelp.status_code)
        
    response_yelp_json = response_yelp.json()
    print("Response JSON:", response_yelp_json)

    pois = response_yelp_json['businesses']
    ratings =[]
    review_counts =[]
    for business in pois:
        ratings.append(business['rating'])
        review_counts.append(business['review_count'])

        temp_df = pd.DataFrame({
            'name':name,
            'rating' : ratings,
            'review count': review_counts,
            'latitude': latitude,
            'longitude': longitude,
            'station_name': stname,
            })
        yelp_df = pd.concat([yelp_df,temp_df], ignore_index=True)
          
JSON(response_fsq_json)  

Response JSON: {'businesses': [{'id': 'Bm4N2Sgc5Q5FHthjmWkvQQ', 'alias': 'castelli-cucina-hamilton', 'name': 'Castelli Cucina', 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/Iqi9gRZTIaV6xUx0pYZHvQ/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/castelli-cucina-hamilton?adjust_creative=cXuAsCWgqLWQR3AJMpZNwQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=cXuAsCWgqLWQR3AJMpZNwQ', 'review_count': 10, 'categories': [{'alias': 'italian', 'title': 'Italian'}, {'alias': 'pizza', 'title': 'Pizza'}], 'rating': 3.5, 'coordinates': {'latitude': 43.26492, 'longitude': -79.86576}, 'transactions': [], 'location': {'address1': '337 James Street N', 'address2': '', 'address3': None, 'city': 'Hamilton', 'zip_code': 'L8L 1H3', 'country': 'CA', 'state': 'ON', 'display_address': ['337 James Street N', 'Hamilton, ON L8L 1H3', 'Canada']}, 'phone': '+12893893065', 'display_phone': '+1 289-389-3065', 'distance': 670.2057251925502}, {'id': '9SSQNmfpLEK_p3otvMVy9Q', '

<IPython.core.display.JSON object>

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [70]:
yelp_df

,name,rating,review count,latitude,longitude,station_name
0,Ferguson at King William,3.5,10,43.269288,-79.871327,Bayfront Park
1,Ferguson at King William,3.5,10,43.269288,-79.871327,Bayfront Park
2,Ferguson at King William,3.5,22,43.269288,-79.871327,Bayfront Park
3,Ferguson at King William,3.5,10,43.269288,-79.871327,Bayfront Park
4,Ferguson at King William,3.5,22,43.269288,-79.871327,Bayfront Park
...,...,...,...,...,...,...
8703,Ferguson at King William,2.0,7,43.255179,-79.861443,Ferguson at King William
8704,Ferguson at King William,4.5,13,43.255179,-79.861443,Ferguson at King William
8705,Ferguson at King William,3.5,21,43.255179,-79.861443,Ferguson at King William
8706,Ferguson at King William,3.5,4,43.255179,-79.861443,Ferguson at King William


In [ ]:
import json

data = response_fsq_json
poi_list = data['results']
for index, row in dfs.iterrows():
    name = row['station_name']
    latitude = row['latitude']
    longitude = row['longitude']
    print(f'{latitude},{longitude}') 
    
    for poi in poi_list:
        name = poi['name']
        rating = poi.get('rating')
        location = poi.get('location',{})
        link = poi.get('link','')
        review_count = poi.get('review_count')
    

    # Process the extracted details as needed
    print(f"Name: {name}")
    print(f"Rating: {rating}")
    print(f"Location: {location}")
    print(f"Link: {link}")
    print(f"Review Count: {review_count}")

Put your parsed results into a DataFrame

In [ ]:
I parsed results into a DataFrame in step one.

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

I didn't get the rating from Foursquare data, so I jsut use Yelp API.

Get the top 10 restaurants according to their rating

In [ ]:
top_10_names = yelp_df.sort_values('rating', ascending=False)['name'].unique()[:10]
print(top_10_names)